In [ ]:
import os
import pandas as pd

# Set default plotly template
import plotly.io as pio
pio.templates.default = "plotly_white"

import plotly.express as px

STATE = 'thermal'
IMG_WIDTH = 600
IMG_HEIGHT = 350

# Set default plotly image format and dimensions
pio.kaleido.scope.default_format = "pdf"
#pio.kaleido.scope.mathjax= None
pio.kaleido.scope.default_width = IMG_WIDTH
pio.kaleido.scope.default_height = IMG_HEIGHT

# Create saving folder if it does not exists
if not os.path.exists(f'../img/obs_evolution/{STATE}'):
    os.makedirs(f'../img/obs_evolution/{STATE}')

In [ ]:
df = pd.read_pickle(f'../objects/{STATE}/observables.pkl')
df = df[df['Time'] <= 1500]

## Mean Photon Numbers

In [ ]:
df_melt = df.melt(id_vars=['Time', 'Interaction Time'], value_vars=['N1', 'N2'], var_name='System', value_name='Temperature')
new_label_s1 = ['S_1$' for _ in range(len(df_melt.loc[df_melt['System'] == 'N1']))]
new_label_s2 = ['S_2$' for _ in range(len(df_melt.loc[df_melt['System'] == 'N2']))]
new_label_dt = [f'${dt}' for dt in df_melt['Interaction Time']]
df_melt.loc[df_melt['System'] == 'N1', 'System'] = new_label_s1
df_melt.loc[df_melt['System'] == 'N2', 'System'] = new_label_s2
df_melt['Interaction Time'] = new_label_dt
fig_temp = px.line(df_melt, x='Time', y='Temperature',
                   color='Interaction Time', line_dash='System',
                   width=IMG_WIDTH, height=IMG_HEIGHT,
                   labels={'Temperature': 'Mean Photon Number',
                           'Time': 'Time (a.u.)',
                           },
                   log_x=False)
fig_temp.update_layout(legend=dict(
                        yanchor="top",
                        y=0.99,
                        xanchor="right",
                        x=0.99,
))
fig_temp.show()

In [ ]:
fig_temp.write_image(f'../img/obs_evolution/{STATE}/temp_evolution.pdf')

## Entropies

In [ ]:
df_melt = df.melt(id_vars=['Time', 'Interaction Time'], value_vars=['S1', 'S2'], var_name='System', value_name='Entropy')
new_label_s1 = ['S_1$' for _ in range(len(df_melt.loc[df_melt['System'] == 'S1']))]
new_label_s2 = ['S_2$' for _ in range(len(df_melt.loc[df_melt['System'] == 'S2']))]
new_label_dt = [f'${dt}' for dt in df_melt['Interaction Time']]
df_melt.loc[df_melt['System'] == 'S1', 'System'] = new_label_s1
df_melt.loc[df_melt['System'] == 'S2', 'System'] = new_label_s2
df_melt['Interaction Time'] = new_label_dt
fig_entropy = px.line(df_melt, x='Time', y='Entropy',
                      color='Interaction Time', line_dash='System',
                      width=IMG_WIDTH, height=IMG_HEIGHT,
                      labels={'Time': 'Time (a.u.)'},
                    )
fig_entropy.update_layout(
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99,
        bordercolor="Black",
        borderwidth=0,
))
fig_entropy.show()

## Quantum Discord

In [ ]:
fig_discord = px.line(df, x='Time', y='Quantum Discord', color='Interaction Time',
                      width=IMG_WIDTH, height=IMG_HEIGHT,
                      labels={'Time': 'Time (a.u.)'})
fig_discord.for_each_trace(lambda t: t.update(name = f'${t.name}$'))
fig_discord.update_layout(
    legend=dict(
        yanchor="bottom",
        y=0.1,
        xanchor="right",
        x=0.99,
        bordercolor="Black",
        borderwidth=0,
))
fig_discord.show()

In [ ]:
fig_discord.write_image(f'../img/obs_evolution/{STATE}/discord_evolution.pdf')

## Mutual Information

In [ ]:
fig_mutinf = px.line(df, x='Time', y='Mutual Information', color='Interaction Time',
                     width=IMG_WIDTH, height=IMG_HEIGHT)
fig_mutinf.for_each_trace(lambda t: t.update(name = f'${t.name}$'))
fig_mutinf.update_layout(
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99,
        bordercolor="Black",
        borderwidth=0,
    )
)
fig_mutinf.show()

In [ ]:
fig_mutinf.write_image(f'../img/obs_evolution/{STATE}/mutinf_evolution.pdf')

## Logarithmic Negativity

In [ ]:
fig_logneg = px.line(df, x='Time', y='Logarithmic Negativity', color='Interaction Time',
                     width=IMG_WIDTH, height=IMG_HEIGHT)
fig_logneg.for_each_trace(lambda t: t.update(name = f'${t.name}$'))
fig_logneg.update_layout(
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99,
        bordercolor="Black",
        borderwidth=0,
    )
)
fig_logneg.show()

In [ ]:
fig_logneg.write_image(f'../img/obs_evolution/{STATE}/logneg_evolution.pdf')

## Heat Exchanges

In [ ]:
df_melt = df.loc[df['Time']>0].melt(id_vars=['Time', 'Interaction Time'], value_vars=['Jc', 'J1', 'J2'], var_name='Current', value_name='Heat')
new_label_j1 = ['J_1$' for _ in range(len(df_melt.loc[df_melt['Current'] == 'J1']))]
new_label_j2 = ['J_2$' for _ in range(len(df_melt.loc[df_melt['Current'] == 'J2']))]
new_label_j = ['J_c$' for _ in range(len(df_melt.loc[df_melt['Current'] == 'Jc']))]
new_label_dt = [f'${dt}' for dt in df_melt['Interaction Time']]
df_melt.loc[df_melt['Current'] == 'J1', 'Current'] = new_label_j1
df_melt.loc[df_melt['Current'] == 'J2', 'Current'] = new_label_j2
df_melt.loc[df_melt['Current'] == 'Jc', 'Current'] = new_label_j
df_melt['Interaction Time'] = new_label_dt
fig_heat = px.line(df_melt, x='Time', y='Heat',
                   color='Interaction Time', line_dash='Current',
                   width=IMG_WIDTH, height=IMG_HEIGHT,
                   labels={'Time': 'Time (a.u.)'})
fig_heat.update_layout(
    legend=dict(
        yanchor="top",
        y=0.95,
        xanchor="right",
        x=0.99,
        bordercolor="Black",
        borderwidth=0,
    ),
    margin=dict(
        autoexpand=False,
    ),
)
fig_heat.update_xaxes(range=[0, 150])
fig_heat.update_yaxes(range=[-0.01, 0.0001])
fig_heat.show()

In [ ]:
fig_heat.write_image(f'../img/obs_evolution/{STATE}/heat_evolution.pdf')